# Deploy a Serverless Model Server with Nuclio-KFServing
  --------------------------------------------------------------------

The following notebook demonstrates how to deploy an XGBoost model using nuclio + KFServing (a.k.a <b>Nuclio-serving</b>)

#### **notebook how-to's**
* Write and test model serving (KFServing like) class in a notebook.
* Deploy the model server as a Nuclio-serving function.
* Invoke and test the serving function.

<a id='top'></a>
#### **steps**
**[define a new function and its dependencies](#define-function)**<br>
**[test the model serving class locally](#test-locally)**<br>
**[deploy our serving class using as a serverless function](#deploy)**<br>
**[test our model server using HTTP request](#test-model-server)**<br>

In [1]:
# mlrun: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio

<a id='define-function'></a>
### **define a new function and its dependencies**

In [2]:
# specify function metadata
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=XGBoostModel

%nuclio: setting kind to 'nuclio:serving'
%nuclio: setting 'MODEL_CLASS' environment variable


In [3]:
%%nuclio cmd
pip install numpy
pip install xgboost

In [4]:
%nuclio config spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


In [5]:
import os
import numpy as np
import xgboost as xgb
from mlrun.runtimes import MLModelServer

In [6]:
class XGBoostModel(MLModelServer):
    def load(self):
        # this is called once to load the model
        # get_model returns file path (copied to local) and extra data dict (of key: DataItem)
        # model object can be accessed at self.model_spec (after running .get_model)
        model_file, _ = self.get_model(".bst")
        self._booster = xgb.Booster(model_file=model_file)

    def predict(self, body):
        try:
            # Use of list as input is deprecated see https://github.com/dmlc/xgboost/pull/3970
            events = np.array(body["instances"])
            dmatrix = xgb.DMatrix(events)
            result: xgb.DMatrix = self._booster.predict(dmatrix)
            return result.tolist()
        except Exception as exc:
            raise Exception(f"Failed to predict {exc}")

The following end-code annotation tells ```nuclio``` to stop parsing the notebook from this cell. _**Please do not remove this cell**_:

In [7]:
# mlrun: end-code

______________________________________________

<a id='test-locally'></a>
### **test the model serving class locally**
The class above can be tested locally. Just instantiate the class, `.load()` will load the model to a local dir.

> **Verify there is a `model.bst` file in the model_dir path (generated by the training notebook)**

In [20]:
# a valist model.bst file MUST EXIST in the model dir
# model_dir = os.path.abspath('./')
model_dir = "/User/mlrun/kfp/032e6d59-6bfe-4ee7-bcf6-1fb26e5db550/1"  # /model.bst'

In [21]:
my_server = XGBoostModel("my-model", model_dir=model_dir)
my_server.load()


We can use the `.predict(body)` method to test the model.

In [25]:
my_server.predict({"instances": [[5], [10]]})

[22:34:39] WARNING: /workspace/src/learner.cc:979: Number of columns does not match number of features in booster. Columns: 1 Features: 4


[[0.7845662832260132,
  0.02555863931775093,
  0.03296218067407608,
  0.0224161297082901,
  0.0224161297082901,
  0.0224161297082901,
  0.0224161297082901,
  0.0224161297082901,
  0.0224161297082901,
  0.0224161297082901],
 [0.7845662832260132,
  0.02555863931775093,
  0.03296218067407608,
  0.0224161297082901,
  0.0224161297082901,
  0.0224161297082901,
  0.0224161297082901,
  0.0224161297082901,
  0.0224161297082901,
  0.0224161297082901]]

<a id='deploy'></a>
### **deploy our serving class using as a serverless function**
in the following section we create a new model serving function which wraps our class , and specify model and other resources.

the `models` dict store model names and the associated model **dir** URL (the URL can start with `S3://` and other blob store options), the faster way is to use a shared file volume, we use `.apply(mount_v3io())` to attach a v3io (iguazio data fabric) volume to our function. By default v3io will mount the current user home into the `\User` function path.

**verify the model dir does contain a valid `model.bst` file**

In [11]:
from mlrun import new_model_server, mount_v3io
import requests

In [12]:
fn = new_model_server(
    "iris-srv", models={"iris_v1": model_dir}, model_class="XGBoostModel"
)

# use mount_v3io() for iguazio volumes or mount_pvc() for k8s PVC volumes
fn.apply(mount_v3io())

In [13]:
# deploy as a serverless function
# specify dashboard=<url> when deploying to a remote cluster
addr = fn.deploy()

[mlrun] 2020-06-08 22:14:47,883 deploy started
[nuclio] 2020-06-08 22:15:31,375 (info) Build complete
[nuclio] 2020-06-08 22:15:40,911 (info) Function deploy complete
[nuclio] 2020-06-08 22:15:40,917 done creating default-iris-srv, function address: 18.221.106.241:31687


<a id="test-model-server"></a>
### **test our model server using HTTP request**


We invoke our model serving function using test data, the data vector is specified in the `instances` attribute.

In [14]:
# KFServing protocol event
event_data = {"instances": [[5], [10]]}

In [ ]:
import json

resp = requests.put(addr + "/iris_v1/predict", json=json.dumps(event_data))
print(resp.text)

**[back to top](#top)**